In [1]:
from geoclip.train import train
from geoclip.train.dataloader import GeoDataLoader
from geoclip.model import GeoCLIP
from datasets import load_dataset, Image
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
def img_train_transform():
    train_transform_list = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    return train_transform_list

def img_val_transform():
    val_transform_list = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(torch.float),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    return val_transform_list

In [3]:
# dataset = load_dataset('yunusserhat/random_streetview_images', split='train[:1%]')

# # drop country_iso_alpha2, address columns
# dataset = dataset.remove_columns(['country_iso_alpha2', 'address'])

# # Add a column called gps that is a tuple of latitude and longitude
# dataset = dataset.map(lambda x: {'gps': (x['latitude'], x['longitude'])})

# dataset = dataset.remove_columns(['latitude', 'longitude'])

# def transform_to_tensor(examples):
#     print(type(examples['image']))
#     examples['image'] = [transforms.PILToTensor()(image) for image in examples['image']]
#     return examples

# dataset.set_transform(transform_to_tensor)

# # dataloader = GeoDataLoader('stochastic/random_streetview_images_pano_v0.0.2', split='train', transform=img_train_transform())
# dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [4]:
dataset = GeoDataLoader('dataset/2k_random_test/1_2k_random_test.csv', 'dataset/2k_random_test', transform=img_train_transform())

Loading image paths and coordinates: 2000it [00:00, 46000.26it/s]


In [5]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
model = GeoCLIP(from_pretrained=False)

/Users/kshitij/miniconda3/envs/mlp/lib/python3.12/site-packages/geoclip/model/location_encoder.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torc

In [7]:
optim = torch.optim.Adam(model.parameters(), lr=3e-5, weight_decay=1e-6)

In [8]:
train(dataloader, model, batch_size=64, device='mps', optimizer=optim, epoch=1)

Starting Epoch 1


  0%|          | 0/32 [00:00<?, ?it/s]

32


  0%|          | 0/32 [00:00<?, ?it/s]

torch.Size([64, 2])
torch.Size([64, 2])
torch.Size([128, 2])
torch.Size([64, 3, 224, 224]) torch.Size([128, 2])


RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same